In this base notebook, we've defined the `fetch_metadata` function that fetches metadata from the specified table in Azure SQL Database and returns it as a list of dictionaries. You can import this base notebook into your other notebooks using the `%run` magic command and use the utility functions across your data processing framework in Azure Databricks.

This notebook also includes utility functions for logging (`log`), error handling (`handle_error`), and generating Spark DataFrames based on metadata (`generate_dataframe`). You can import this base notebook into your other notebooks using the `%run` magic command and use these utility functions across your data processing framework in Azure Databricks.

In [ ]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
import pandas as pd
import pyodbc
import json
import logging

# Initialize the Spark session
spark = SparkSession.builder.getOrCreate()

# Set up logging configuration
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Define the function to fetch metadata from Azure SQL Database
def fetch_metadata(jdbc_url, jdbc_user, jdbc_password, table_name):
    connection_string = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={jdbc_url};UID={jdbc_user};PWD={jdbc_password}"
    
    with pyodbc.connect(connection_string) as conn:
        query = f"SELECT * FROM {table_name}"
        metadata_df = pd.read_sql(query, conn)
    
    return metadata_df.to_dict(orient="records")


# Utility function for logging
def log(message, level=logging.INFO):
    logging.basicConfig(level=level)
    logger = logging.getLogger(__name__)
    logger.log(level, message)

# Utility function for error handling
def handle_error(exception, message="An error occured"):
    log(f"{message}: {str(exception)}", level=logging.ERROR)

# Utility function to generate Spark DataFrame based on metadata
def generate_dataframe(data, schema_json):
    schema = StructType.fromJson(json.loads(schema_json))
    return spark.createDataFrame(data, schema)





# Example usage of utility functions

```python
try:
    metadata = fetch_metadata("<jdbc_url>", "<jdbc_user>", "<jdbc_password>", "DataSources")
    log("Fetched metadata successfully")
except Exception as e:
    handle_error(e, "Failed to fetch metadata")
```